In [1]:
from jupyter_set_up import init_django

init_django('pictionary')

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [3]:
class CNN(nn.Module):
    def __init__(self, classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 64, 3, 1, 1)
        self.conv2 = nn.Conv2d(64, 256, 3, 1, 1)
        self.conv3 = nn.Conv2d(256, 512, 3, 1, 1)

        self.fc1 = nn.Linear(512*3*3, 512)
        self.fc2 = nn.Linear(512, classes)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [4]:
def create_X_y(file):
    X = []
    y = []
    for label, img in np.load(file, allow_pickle=True):
        y.append(torch.tensor(int(label)-1))
        X.append(torch.tensor(img))
    return torch.stack(X), torch.stack(y)

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(cnn, train_X, train_y):
    BATCH_SIZE = 500
    EPOCHS = 16
    optimizer = optim.SGD(cnn.parameters(), lr=0.1)
    loss_function = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 28, 28).to(device)
            batch_y = train_y[i:i+BATCH_SIZE].to(device)

            outputs = cnn(batch_X)
            loss = loss_function(outputs, batch_y)
            
            cnn.zero_grad()
            loss.backward()
            optimizer.step()
            
        print("Epoch: {} Loss: {}".format(epoch+1, loss))


def test(cnn, test_X, test_y):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = test_y[i].to(device)
            net_out = cnn(test_X[i].view(-1, 1, 28, 28).to(device))[0]
            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Accuracy: {}".format(round(correct / total, 3)))

In [6]:
train_X, train_y = create_X_y('train.npy')

In [14]:
from PIL import Image

for i in range(-299, -1):
    img = Image.fromarray(np.array(train_X[i]))
    train_X[i] = torch.tensor(np.pad(np.array(img.resize(size=(26, 26))), 1, mode='constant'))

In [26]:
cnn1 = CNN(50).to(device)
train(cnn1, train_X, train_y)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [09:01<00:00,  6.89it/s]


Epoch: 1 Loss: 3.504225015640259


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [09:09<00:00,  6.80it/s]


Epoch: 2 Loss: 2.975191354751587


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [09:10<00:00,  6.79it/s]


...

Epoch: 14 Loss: 1.940956711769104


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [09:13<00:00,  6.75it/s]


Epoch: 15 Loss: 1.9111964702606201


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [09:14<00:00,  6.73it/s]


Epoch: 16 Loss: 1.8835115432739258


In [18]:
test_X, test_y = create_X_y('test.npy')

In [19]:
for i in range(-149, -1):
    img = Image.fromarray(np.array(test_X[i]))
    test_X[i] = torch.tensor(np.pad(np.array(img.resize(size=(26, 26))), 1, mode='constant'))

In [27]:
test(cnn1, test_X, test_y)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 933509/933509 [22:03<00:00, 705.43it/s]

Accuracy: 0.875


In [ ]:
cnn = CNN(40).to(device)
train(cnn, train_X[:-299], train_y[:-299])

 81%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                           | 3013/3734 [07:19<01:45,  6.85it/s]

In [ ]:
test(cnn, test_X[:-149], test_y[:-149])

In [21]:
import pickle
pickle.dump(cnn1,open("cnn_model.sav", "wb"))

In [30]:
model = pickle.load(open("cnn_model.sav", "rb"))
output = model(test_X[-7].view(-1, 1, 28, 28).to(device))[0]
prediction = torch.argmax(output)
int(prediction), int(test_y[-7])

(46, 48)

In [15]:
import pickle
pickle.dump(cnn,open("cnn_model_QD.sav", "wb"))